In [1]:
import rpy2.ipython
%load_ext rpy2.ipython

C:\ProgramData\Anaconda3\lib\site-packages\rpy2\robjects\pandas2ri.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


In [3]:
%R require(ggplot2)

array([1], dtype=int32)

# Pre-processing

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.groupby([df.combo].mean)

In [2]:
# pandas_profiling
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport


#data = pd.read_excel (r'neonatal/neonatal outcomes.xlsx')
df = pd.read_excel('neonatal/neonatal outcomes.xlsx', engine='openpyxl')


profile = ProfileReport(df, title="Pandas Profiling Report")

profile.to_file("output.html")

In [ ]:
import sweetviz as sv
#analyzing the dataset
car_report = sv.analyze(df)
car_report.show_html('Report.html')
#Analyzing a single dataframe (and its -optional- target feature)
my_report = sv.analyze([train, "Train"],target_feat='SalePrice')
my_report.show_html('Report.html')
#Comparing two dataframes (e.g. Test vs Training sets)
my_report1 = sv.compare([train, "Train"], [test, "Test"], "SalePrice")
my_report1.show_html('Report.html')


In [ ]:

import IPython
IPython.display.HTML('Report.html')

In [ ]:
# remove outliners
max_thresold = df['height'].quantile(0.95)
min_thresold = df['height'].quantile(0.05)
df = df[(df['height']<max_thresold) & (df['height']>min_thresold)]

In [ ]:
# FeatureSelector 

# Delelte duplicate features
unique_df = data_t.drop_duplicates(keep='first').T
unique_df.shape

# feature selection
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2
X, y = load_digits(return_X_y=True)
X.shape
(1797, 64)
X_new = SelectKBest(chi2, k=20).fit_transform(X, y)
X_new.shape


# Missing Values
from feature_selector import FeatureSelector
fs = FeatureSelector(data = X, labels = train_labels)
fs.identify_missing(missing_threshold=0.6)
missing_features = fs.ops['missing']
missing_features[:10]
fs.plot_missing()


# Collinear Features
fs.identify_collinear(correlation_threshold=0.975)
correlated_features = fs.ops['collinear']
correlated_features[:5]
fs.plot_collinear()
fs.record_collinear.head()

# Zero Importance Features
fs.identify_zero_importance(task = 'classification', eval_metric = 'auc', 
                            n_iterations = 10, early_stopping = True)
fs.plot_feature_importances(threshold = 0.99, plot_n = 12)


In [ ]:
# dimensionality reduction using pca
# البتاع ده بيقلل الvariance
# لازم أختار عدد يحافظ على 95% من الفاريانس على الأقل
data = ...
# define transform
pca = PCA()
# prepare transform on dataset
pca.fit(data)
# apply transform to dataset
transformed = pca.transform(data)
# define the pipeline
steps = [('pca', PCA()), ('m', LogisticRegression())]
model = Pipeline(steps=steps)
# test classification dataset
from sklearn.datasets import make_classification
# define dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=7)
# summarize the dataset
print(X.shape, y.shape)


In [ ]:
# evaluate pca with logistic regression algorithm for classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
# define dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=7)
# define the pipeline
steps = [('pca', PCA(n_components=10)), ('m', LogisticRegression())]
model = Pipeline(steps=steps)
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
#word to num
from word2number import w2n
df.x = x2n.word_to_num(df.x)

In [ ]:
#STEROID(S) One hot encoding
#steroids_dum=pd.get_dummies(df.steroids)
#steroids_dum.columns = ['steroids1', 'steroids2', 'steroids3']
#df_M = pd.concat([df,steroids_dum],axis='columns')

In [ ]:
# Analyze how many null values are in the dataset
import math
#data.isnull().sum()
# data.plot_missing()

# drop all missing values in all columns:
#data = data.dropna()

# Mean value imputation:
# data["Age"].fillna(data["Age"].mean(), inplace=True)

# Check the database after missed data were dropped
#data.shape

In [ ]:
## Correlation
import seaborn as sns
import matplotlib.pyplot as plt
#get correlations of each features in dataset
corrmat = data.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(50,50))
#plot heat map
g=sns.heatmap(data[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
#implanced data
ones = data[data['Class']==1]
zeros = data[data['Class']==0]
print(ones.shape,zeros.shape)
# لو طلع في فرق بين الرقمين


# Implementing Undersampling بياخد عينة عشوائية من المجموعة الكبيرة
nm = NearMiss(random_state=42)
X_res,y_res=nm.fit_sample(X,Y)

# Implementing Oversampling (شبه البوتستراب)
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss
smk = SMOTETomek(random_state=42)
X_res,y_res=smk.fit_sample(X,Y)

In [ ]:
#Composite Outcome
#C_outcome = df['neonatal_morbidity'] + df ['perinatal_death'] + df['nicu_admission'] + df['resp_support'] + df['apgar_5']
#C_outcome = C_outcome.replace([2,3,4,5],1)
#C_outcome.columns = ['C_outcome']

In [ ]:
# some algorithms work better with normalization
Features =  np.asarray(df_final[['age', 'parity', 'bmi', 'smoking',
                           'race3', 'race4', 
                           'prev_cs', 'ivf', 'conception_type','gdm', 'pregdm', 'htn', 'ghtn', 'chtn', 'pree', 'iugr',
                           'gyn_dc', 'ob_dc', 'ga_diagnosis', 'method1', 'method2', 'method3', 
                           'ga_deliver', 'steroid',
                           'pre_hgb', 'site1', 'site2', 'site3', 'site4', 'site5', 
                           'incision1', 'incision2', 'incision3',  
                           'incision_placenta', 'pre_us', 'intra_us',
                           'organ_invasion', 
                           'cord_clamp', 'planned']])

# Name every outcome differently